In [1]:
import json
import os
import sys
import cProfile
import pstats

import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error

sys.path.append(r"../")
from AssistanceTransform import transform, estimator

# Performance
In this Notebook, I will be performance testing various parts of the AssistanceTransform library. This includes both speed and accuracy testing.

First getting the testing data ready:

In [2]:
image_coord_dict = dict()
image_coord_dict["00.jpg"] = np.array([[1470, 1430], [2333, 1355], [3247, 1781], [1935, 1952]])
image_coord_dict["01.jpg"] = np.array([[1495, 1552], [2219, 1589], [1843, 1969], [805, 1875]])
image_coord_dict["03.jpg"] = np.array([[1216, 1398], [2215, 1754], [3268, 1530], [2067, 1282]])   

def setup_vars():
    """Loads data for test_transform_image"""
    data_dir = r"./data/table"
    json_fp = os.path.join(data_dir, "anno.json")
    arr_fp = os.path.join(data_dir, "anno.npz")
    with open(json_fp, "r") as fp:
        mappings = json.load(fp)

    with np.load(arr_fp) as arrs:
        anno_dict = {img: {"heads": arrs[f"{prefix}heads"],
                           "feet": arrs[f"{prefix}feet"]}
                     for img, prefix in mappings.items()}
    
    for key, items in anno_dict.items():
        if key.endswith("02.jpg"):
            continue
        else:
            image_coords = image_coord_dict[key[-6:]]
        # feet and heads have been swapped in annotations
        reference = np.array([items["feet"], items["heads"]])
        height = 0.095  # m
        STD = 0.01  # m
        img = Image.open(key)
        yield (img, reference, height, STD, image_coords)

## Speed

In [3]:
generator = setup_vars()
pr = cProfile.Profile()
pr.enable()
coords_sq = (transform.transform_image(*params, iters=1e5)[:, :2] for params in generator)
y_pred = np.array([estimator.area(poly) for poly in coords_sq])
pr.disable()
pr.print_stats("tottime")

C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [01:02<00:00, 1600.78it/s, acc_rate=0.392, factor=0.103]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0

       elevation_m   tilt_deg  heading_deg   roll_deg  probability
0         0.594352  73.764421   -67.754553 -10.558436    12.151129
1         0.554278  73.674993   -67.717094 -10.626573     9.022700
2         0.554278  73.674993   -67.717094 -10.626573     9.022700
3         0.554278  73.674993   -67.717094 -10.626573     9.022700
4         0.722995  73.713561   -67.725940 -10.662976    12.580697
...            ...        ...          ...        ...          ...
89994     0.701389  71.791669   -72.822327  -6.846265    13.965606
89995     0.701389  71.791669   -72.822327  -6.846265    13.965606
89996     0.720966  71.768016   -72.745766  -6.903810    14.118340
89997     0.720966  71.768016   -72.745766  -6.903810    14.118340
89998     0.751312  71.542432   -72.559509  -6.793434    14.007773

[89999 rows x 5 columns]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [01:03<00:00, 1586.36it/s, acc_rate=0.424, factor=0.133]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|▏                                                                                                                                      | 177/100000 [00:00<00:56, 1767.70it/s, acc_rate=0.0448, factor=0.5]

       elevation_m   tilt_deg  heading_deg   roll_deg  probability
0         0.948633  70.229950   -74.482620  -3.528506    11.945766
1         0.831185  70.197288   -74.560114  -3.672276    13.312985
2         0.831185  70.197288   -74.560114  -3.672276    13.312985
3         0.831185  70.197288   -74.560114  -3.672276    13.312985
4         0.831185  70.197288   -74.560114  -3.672276    13.312985
...            ...        ...          ...        ...          ...
89994     1.435945  53.554102   -54.252202 -30.288151    11.858250
89995     1.435945  53.554102   -54.252202 -30.288151    11.858250
89996     1.435945  53.554102   -54.252202 -30.288151    11.858250
89997     1.435945  53.554102   -54.252202 -30.288151    11.858250
89998     1.491999  53.573941   -54.527583 -30.327749    11.453277

[89999 rows x 5 columns]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [01:02<00:00, 1604.42it/s, acc_rate=0.328, factor=0.234]


...

     1299    0.000    0.000    0.000    0.000 format.py:393(len)
       15    0.000    0.000    0.008    0.001 _distn_infrastructure.py:711(_construct_doc)
        1    0.000    0.000  188.174  188.174 <ipython-input-3-6437586f9fa3>:5(<listcomp>)
     1132    0.000    0.000    0.000    0.000 {method 'keys' of 'collections.OrderedDict' objects}
      195    0.000    0.000    0.000    0.000 config.py:557(_get_root)
        8    0.000    0.000    0.004    0.001 format.py:558(_read_array_header)
  190/163    0.000    0.000    0.009    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
       45    0.000    0.000    0.000    0.000 inspect.py:1799(_signature_bound_method)
       13    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap_external>:1356(find_spec)
       40    0.000    0.000    0.001    0.000 zipfile.py:907(read)
     1683    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
      590    0.000    0.000    0.000    0.000 sre_parse.py:233(__nex

       15    0.000    0.000    0.000    0.000 sre_compile.py:276(_optimize_charset)
      375    0.000    0.000    0.000    0.000 inspect.py:2835(parameters)
       33    0.000    0.000    0.000    0.000 common.py:1025(is_datetime_or_timedelta_dtype)
        9    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap_external>:1240(_get_spec)
       24    0.000    0.000    0.000    0.000 series.py:492(name)
      150    0.000    0.000    0.000    0.000 format.py:1777(_is_number)
       18    0.000    0.000    0.000    0.000 frame.py:3179(_box_col_values)
        3    0.000    0.000    0.010    0.003 format.py:817(write_result)
       45    0.000    0.000    0.000    0.000 TiffImagePlugin.py:643(<lambda>)
      221    0.000    0.000    0.000    0.000 {method 'isidentifier' of 'str' objects}
       24    0.000    0.000    0.000    0.000 series.py:442(name)
      120    0.000    0.000    0.000    0.000 inspect.py:158(isfunction)
      126    0.000    0.000    0.000    0.000 enum.py

       15    0.000    0.000    0.000    0.000 _util.py:191(check_random_state)
       27    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:51(_r_long)
       71    0.000    0.000    0.000    0.000 sre_parse.py:249(match)
        8    0.000    0.000    0.001    0.000 tokenize.py:312(untokenize)
        9    0.000    0.000    0.000    0.000 managers.py:683(_consolidate_check)
       47    0.000    0.000    0.000    0.000 sre_parse.py:286(tell)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(all)
       54    0.000    0.000    0.000    0.000 _binary.py:70(i16be)
       45    0.000    0.000    0.000    0.000 _util.py:339(<dictcomp>)
       17    0.000    0.000    0.000    0.000 zipfile.py:1935(_fpclose)
       18    0.000    0.000    0.000    0.000 _collections_abc.py:72(_check_methods)
        1    0.000    0.000    0.000    0.000 contextlib.py:481(__exit__)
        6    0.000    0.000    0.000    0.000 indexing.py:1500(_get_slice_ax

        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:884(__init__)
        9    0.000    0.000    0.000    0.000 managers.py:684(<listcomp>)
        6    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
        2    0.000    0.000    0.000    0.000 tokenize.py:380(find_cookie)
        3    0.000    0.000    0.000    0.000 _binary.py:50(i32le)
        3    0.000    0.000    0.000    0.000 format.py:883(<listcomp>)
       15    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_object}
       16    0.000    0.000    0.000    0.000 {method 'seekable' of '_io.BufferedReader' objects}
        1    0.000    0.000    0.000    0.000 decoder.py:332(decode)
        1    0.000    0.000    0.000    0.000 context.py:69(RLock)
       45    0.000    0.000    0.000    0.000 {built-in method sys.getrecursionlimit}
        9    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:93(_path_isfile)
       36    0.000    0.000    0.000    0

        5    0.000    0.000    0.000    0.000 TiffImagePlugin.py:629(decorator)
        3    0.000    0.000    0.000    0.000 {method 'join' of 'bytes' objects}
        3    0.000    0.000    0.000    0.000 interactiveshell.py:705(get_ipython)
        1    0.000    0.000    0.000    0.000 npyio.py:222(__del__)
        1    0.000    0.000    0.000    0.000 synchronize.py:90(_make_methods)
        3    0.000    0.000    0.000    0.000 common.py:1293(<lambda>)
        6    0.000    0.000    0.000    0.000 fractions.py:278(denominator)
        5    0.000    0.000    0.000    0.000 {built-in method _sre.unicode_tolower}
        6    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
        6    0.000    0.000    0.000    0.000 {built-in method builtins.vars}
        4    0.000    0.000    0.000    0.000 typing.py:657(__eq__)
        3    0.000    0.000    0.000    0.000 base.py:3892(_get_engin

## Accuracy
As the "real" positions in images currently aren't known, in order to figure out how close CameraTransform is to "real" positions, we have to take a derivative of these positions. For the purposes of this assignment, I've chosen the area of the polygon, as this is an important variable in the use case.

The three images used for testing all feature a table with a area of `133` cm$^{2}$. 

In [4]:
y_true = np.repeat(1.33455, y_pred.size)

In [5]:
y_pred

array([1.32968404, 1.48072635, 1.5110926 ])

In [6]:
mean_squared_error(y_true, y_pred, squared=False)

0.13236124032448218

An RMSE of around 20 square centimeters is adequate. Especially the latter two images are taken and worse angles and are insufficiently labeled in comparison to `img_03.jpg`.